In [ ]:
import numpy
import theano
import theano.tensor as T
import lasagne
from sklearn.metrics import precision_recall_fscore_support
import os
import timeit
import pickle
from hyperopt import STATUS_OK
from sklearn.metrics import accuracy_score

#### Build the Neural Network using Lasagne model i.e defines architecture of Neural Network.

In [ ]:
def build_nn(input_var, mask, num_features, num_lstm_layers, num_lstm_units,
             num_dense_layers, num_dense_units):

    l_input = lasagne.layers.InputLayer(shape=(None, None, num_features),
                                        input_var=input_var)

    l_mask = lasagne.layers.InputLayer(shape=(None, None), input_var=mask)

    l_lstm1 = lasagne.layers.LSTMLayer(
                    l_input, 
                    num_units=num_lstm_units,
                    mask_input=l_mask,
                    peepholes=False,
                    forgetgate=lasagne.layers.Gate(
                        W_in=lasagne.init.Normal(0.1),
                        W_hid=lasagne.init.Normal(0.1),
                        W_cell=lasagne.init.Normal(0.1),
                        b=lasagne.init. Constant(2.),
                        nonlinearity=lasagne.nonlinearities.sigmoid))
    for nlstm in range(num_lstm_layers-1):
        l_lstm2 = lasagne.layers.LSTMLayer(
                  l_lstm1,
                  num_units=num_lstm_units,
                  mask_input=l_mask, peepholes=False,
                  forgetgate=lasagne.layers.Gate(
                        W_in=lasagne.init.Normal(0.1),
                        W_hid=lasagne.init.Normal(0.1),
                        W_cell=lasagne.init.Normal(0.1),
                        b=lasagne.init. Constant(2.),
                        nonlinearity=lasagne.nonlinearities.sigmoid))
        l_lstm1 = l_lstm2

    l_shp = lasagne.layers.ReshapeLayer(l_lstm1, (-1, num_lstm_units))

    l_in_drop = lasagne.layers.DropoutLayer(l_shp, p=0.2)

    l_hidden1 = lasagne.layers.batch_norm(lasagne.layers.DenseLayer(
                                  l_in_drop,
                                  num_units=num_dense_units,
                                  W=lasagne.init.Normal(0.1),
                                  b=lasagne.init.Normal(0.1),
                                  nonlinearity=lasagne.nonlinearities.rectify))

    for nl in range(num_dense_layers-1):
        l_hidden2 = lasagne.layers.batch_norm(lasagne.layers.DenseLayer(
                                  l_hidden1,
                                  num_units=num_dense_units,
                                  W=lasagne.init.Normal(0.1),
                                  b=lasagne.init.Normal(0.1),
                                  nonlinearity=lasagne.nonlinearities.rectify))
        l_hidden1 = l_hidden2

    l_out_drop = lasagne.layers.DropoutLayer(l_hidden1, p=0.5)

    l_softmax = lasagne.layers.DenseLayer(
                                   l_out_drop,
                                   num_units=4,
                                   W=lasagne.init.Normal(0.1),
                                   b=lasagne.init.Normal(0.1),
                                   nonlinearity=lasagne.nonlinearities.softmax)

    return (l_softmax)

#### splits training data into mini-batches and returns iterator

In [ ]:
def iterate_minibatches(inputs, mask, rmdoublemask, targets,
                        batchsize, max_seq_len=25, shuffle=False):
    
    # print "Before: target size:", targets.size
    # print "Before: len of targets", len(targets)
    # print "Before: len of batchsize", batchsize
    # print "Before: Len of inputs:", len(inputs)
    # print "Before: Len of rmdoublemask:", len(rmdoublemask)
    # print "Before: Size of rmdoublemask:", rmdoublemask.size
    # targets = numpy.reshape(targets, (-1, max_seq_len))
    # rmdoublemask = numpy.reshape(rmdoublemask, (-1, max_seq_len))
    rmdoublemask = numpy.reshape(rmdoublemask, (len(targets), -1))
    # print "Len of inputs:", len(inputs)
    # print "len of targets", len(targets)
    assert len(inputs) == len(targets)
    if shuffle:
        indices = numpy.arange(len(inputs))
        numpy.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        newtargets = targets[excerpt].flatten()
        newrmdoublemask = rmdoublemask[excerpt].flatten()
        yield inputs[excerpt], mask[excerpt], newrmdoublemask, newtargets
#%%